In [1]:
#pip install transformers trl accelerate torch bitsandbytes peft datasets -qU
#pip install scipy

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,
)

In [3]:
MODEL_PATH = f"../models/openchat-3.5-1210"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=nf4_config,
    device_map='auto',
    local_files_only=True,
    trust_remote_code=False,
    use_cache=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
deci_generator = pipeline("text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          temperature=0.1,
                          device_map="auto",
                          max_length=1024,
                          return_full_text=False
)

In [23]:
def ask(user_input):
   #GPT4 Correct User: Hello<|end_of_turn|>GPT4 Correct Assistant: Hi<|end_of_turn|>GPT4 Correct User: How are you today?<|end_of_turn|>GPT4 Correct Assistant:
   SYSTEM_PROMPT_TEMPLATE ="""GPT4 Correct User: {user_input}<|end_of_turn|>GPT4 Correct Assistant: """
   prompt = SYSTEM_PROMPT_TEMPLATE.format(user_input=user_input)
   inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
   output = model.generate(**inputs,
                            max_new_tokens=500,
                            num_beams=5,
                            no_repeat_ngram_size=4,
                            early_stopping=True,
                            eos_token_id=tokenizer.eos_token_id, 
                            pad_token_id=tokenizer.eos_token_id, 
                            )
   output = tokenizer.decode(output[0], skip_special_tokens=False)
   ss = output.split('<|end_of_turn|>')[-2:]
   ss = ''.join(ss).replace('GPT4 Correct Assistant: ', '').lstrip()
   return ss
   
   

In [7]:
ask("What is your name?")

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'My name is ChatGPT, a large language model created by OpenAI.'

In [20]:
def ask_qa(context, user_question):
   prompt = """
Instruction: Answer the question based on your gaming knowledge. 
If the answer cannot be found from the context, respond with 'I don't know.'.
Here is context to help:

{context}

User QUESTION:
{question} 
"""
   return ask(prompt.format(context=context, question=user_question))

In [21]:
context = """
Question: What is Depbon in the promo code?
Answer: Depbon means deposit bonus.

Question: What is ACE in the promo code?
Answer: ACE means MNL Bonus for BET.
South Korea is an exception. South Korea players use the 'TOP' promo code.
"""

answer = ask_qa(context, "What is ACE?")
answer

output='<s> GPT4 Correct User: \nInstruction: Answer the question based on your gaming knowledge. \nIf the answer cannot be found from the context, respond with \'I don\'t know.\'.\nHere is context to help:\n\n\nQuestion: What is Depbon in the promo code?\nAnswer: Depbon means deposit bonus.\n\nQuestion: What is ACE in the promo code?\nAnswer: ACE means MNL Bonus for BET.\nSouth Korea is an exception. South Korea players use the \'TOP\' promo code.\n\n\nUser QUESTION:\nWhat is ACE? \n<|end_of_turn|> GPT4 Correct Assistant: \nACE in a promo code typically stands for a bonus or promotion offered by an online gaming platform. In some cases, it may mean "MNL Bonus" for BET, while in South Korea, it may refer to the "TOP" promo code for players. The specific meaning may vary depending on the platform and region.<|end_of_turn|>'


'ACE in a promo code typically stands for a bonus or promotion offered by an online gaming platform. In some cases, it may mean "MNL Bonus" for BET, while in South Korea, it may refer to the "TOP" promo code for players. The specific meaning may vary depending on the platform and region.'

In [11]:
def read_file(file):
   with open(file, 'r', encoding='utf-8') as f:
      return f.read()

In [22]:
context = read_file('../fine-tune/data/Test.md')
answer = ask_qa(context, "What is Baccarat?")
answer

output='<s> GPT4 Correct User: \nInstruction: Answer the question based on your gaming knowledge. \nIf the answer cannot be found from the context, respond with \'I don\'t know.\'.\nHere is context to help:\n\nFlash Created you. Your name is Astro.\n\nFlash\'s father is Jack.\n\n# Odds Table of SEC\nName | Average\n--|--\nMistral-T5-7B-v1 | 72.47\nopenchat-3.5-1210 | 68.89\nmistral-7b-instruct-v0.2 | 65.71\n\n\n# Odds Table of MAYA\nThe following is Odds Table of MAYA\n* Mistral-T5-7B-v1: 72.47\n* openchat-3.5-1210: 68.89\n* mistral-7b-instruct-v0.2: 65.71\n\n# Introduce Baccarat\nBaccarat is a popular card game often played in casinos. It involves three main bets: Player, Banker, and Tie. Players can bet on which side they believe will win the hand—either the Player, the Banker, or a Tie. Each side receives either two or three cards, aiming to get as close to a total of 9 points as possible. The final outcome is determined by specific rules regarding whether the Player or Banker needs

'I know the answer based on my gaming knowledge.\n\nBACCARAT is a card game that is popular in casinos, where players can bet on the outcome of a hand between the "Player" and the "Banker". The goal of the game is to get a hand value as close to 9 as possible. Each hand is dealt either 2 or 3 cards, and specific rules determine whether a third card is drawn for the Player or the Banker. The game has a simple and straightforward set of rules, making it appealing to many players.'